In [ ]:
"""
Created on Mon Jul 24 11:36 2020

This is a script to cut out the T and S in the 50 km in front of the ice front

@author: Clara Burgard
"""

- calculate the distance to the ice front for the small domain in front of the ice shelf
- take the ocean points at distance of ~50 km of the ice front 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

import itertools

import distributed
import glob

from tqdm.notebook import tqdm


In [ ]:
client = distributed.Client(n_workers=8, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN THE DATA

In [ ]:
mod = 'UKESM1-0-LL' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
    ens_run = 'r1i1p1f2'
    to2300 = False
elif mod in ['GISS-E2-1-H']:
    ens_run = 'r1i1p1f2'
    to2300 = True
elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
    ens_run = 'r1i1p1f1'
    to2300 = True
elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
    ens_run = 'r1i1p1f1'
    to2300 = False
elif mod == 'UKESM1-0-LL':
    ens_run = 'r4i1p1f2'
    to2300 = True     
elif mod == 'CESM2':
    ens_run = 'r11i1p1f1'
    to2300 = False        


if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
elif scenario == 'ssp245':
    yystart = 2015
    yyend = 2100  
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_8km/'
inputpath_BedMachine='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_ISMIP = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/T_S_PROF/WOA2018_EN4_MEOP/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

COMPUTING THE MEAN PROFILES

CONTINENTAL SHELF

In [ ]:
mask_domain_distkm = 50000

CREATE SUM OF T AND S OVER THE DOMAIN OF INTEREST

In [ ]:
chunk_size = 200

In [ ]:
dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf_8km.nc',chunks={'x': chunk_size, 'y': chunk_size, 'Nisf': 30})

T_ocean_ISMIP = xr.open_mfdataset(inputpath_ISMIP + 'obs_temperature_1995-2017_8km_x_60m.nc', chunks={'x': chunk_size, 'y': chunk_size, 'depth': 50})
S_ocean_ISMIP = xr.open_mfdataset(inputpath_ISMIP + 'obs_salinity_1995-2017_8km_x_60m.nc', chunks={'x': chunk_size, 'y': chunk_size, 'depth': 50})

mask_sum = xr.open_dataset(inputpath_profiles + 'mask_sum.nc', chunks={'x': chunk_size, 'y': chunk_size, 'Nisf': 30})
file_BedMachine_orig = xr.open_mfdataset(inputpath_BedMachine+'BedMachine_v2_aggregated8km_allvars.nc').sel(x=dist_to_front_file.x,y=dist_to_front_file.y).chunk({'x': chunk_size, 'y': chunk_size})
grid_cell_area_file = xr.open_mfdataset(inputpath_BedMachine+'gridarea_ISMIP6_AIS_8000m_grid_Clara.nc').sel(x=dist_to_front_file.x,y=dist_to_front_file.y).chunk({'x': chunk_size, 'y': chunk_size})

dist_to_front = dist_to_front_file['dist_from_front']
ocean_conc = file_BedMachine_orig['ocean_conc']
true_grid_cell_area = grid_cell_area_file['true_grid_cell_area']

In [ ]:
mask_km = dist_to_front <= mask_domain_distkm
ocean_mask = ocean_conc > 0
depth_mask = S_ocean_ISMIP.z >= file_BedMachine_orig['bed']
ocean_and_depth = ocean_mask & depth_mask

In [ ]:
weighted_ocean_mask = (mask_km * ocean_conc * ocean_and_depth * true_grid_cell_area)#.load()

In [ ]:
ds_T_sum = (T_ocean_ISMIP['temperature'] * weighted_ocean_mask).sum(['x','y'])#.load()
ds_T_mean = (ds_T_sum/mask_sum['mask_sum'])

ds_T_mean.to_dataset(name='thetao').to_netcdf(inputpath_profiles + 'T_mean_prof_50km_contshelf_ISMIP.nc')

ds_S_sum = (S_ocean_ISMIP['salinity'] * weighted_ocean_mask).sum(['x','y'])#.load()
ds_S_mean = (ds_S_sum/mask_sum['mask_sum'])

ds_S_mean.to_dataset(name='so').to_netcdf(inputpath_profiles + 'S_mean_prof_50km_contshelf_ISMIP.nc')

In [ ]:
scenario = 'historical'

for mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
           'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','IPSL-CM6A-LR','MPI-ESM1-2-HR',
           'MRI-ESM2-0','UKESM1-0-LL']:
    
    print(mod)
    
    inputpath_data_orig='/bettik/jourdai1/OCEAN_DATA_CMIP6_STEREO/'+mod+'/'

    
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        ens_run = 'r1i1p1f2'
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        ens_run = 'r1i1p1f2'
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        ens_run = 'r1i1p1f1'
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        ens_run = 'r1i1p1f1'
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        ens_run = 'r4i1p1f2'
        to2300 = True     
    elif mod == 'CESM2':
        ens_run = 'r11i1p1f1'
        to2300 = False

    T_ocean_clim = xr.open_mfdataset(inputpath_data_orig+'thetao_Clim_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', use_cftime=True, parallel=True, chunks={'x': chunk_size, 'y': chunk_size, 'depth': 50}).squeeze().drop('time')
    S_ocean_clim = xr.open_mfdataset(inputpath_data_orig+'so_Clim_'+mod+'_'+scenario+'_'+ens_run+'_*.nc', use_cftime=True, parallel=True, chunks={'x': chunk_size, 'y': chunk_size, 'depth': 50}).squeeze().drop('time') #, chunks={'x': 300, 'y': 300, 'depth': 50, 'time': 5})

    ds_T_sum = (T_ocean_clim['thetao'] * weighted_ocean_mask).sum(['x','y'])#.load()
    ds_T_mean = (ds_T_sum/mask_sum['mask_sum'])

    ds_T_mean.to_dataset(name='thetao').to_netcdf(inputpath_profiles + mod + '/T_mean_prof_50km_contshelf_'+mod+'_clim.nc')

    ds_S_sum = (S_ocean_clim['so'] * weighted_ocean_mask).sum(['x','y'])#.load()
    ds_S_mean = (ds_S_sum/mask_sum['mask_sum'])

    ds_S_mean.to_dataset(name='so').to_netcdf(inputpath_profiles + mod + '/S_mean_prof_50km_contshelf_'+mod+'_clim.nc')